# 导入相关库包

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入数据

In [3]:
# cora数据集导入
cora_path = '../dataset/cora/cora/cora.content'
cora = pd.read_csv(cora_path, sep='\t', header=None)
cora.head()

,0,1,2,3,4,5,6,7,8,9,...,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


# GRAPH ATTENTIONAL LAYER

In [ ]:
# 输入是一个二维数组，行数是N，列数是F，表示N个样本，每个样本有F个特征
# 输出也是一个二维数组，行数是N，列数是F1，表示N个样本的标签，F1表示新产生的特征数量
